ЛАБОРАТОРНА РОБОТА 1
====================================================
Стиснення тексту за допомогою генеративної моделі
=====
Автори: Баклан Аліса, Кіндякова Діана, Віткіна Анна
Група МІ-4


In [ ]:
!pip install transformers
!pip torch

ERROR: unknown command "torch"


In [ ]:
import re
from transformers import AlbertTokenizer, GPT2LMHeadModel
import torch

PREDICT_WINDOW = 10000
NUM_WORDS = 100
SHOW_TOKENS = False
SHOW_PREDICTION = False

In [ ]:
class TextCompressor:
    def __init__(self, model_name):
        self.tokenizer = AlbertTokenizer.from_pretrained(model_name)
        self.model = GPT2LMHeadModel.from_pretrained(model_name)

    def compress(self, input_text):
        compressed_tokens = []
        current_text = ""

        # Use regular expression to split the input text into tokens with spaces as separators
        pattern = r'\d+|[а-яА-ЯїЇєЄіIґҐ]+\'?[а-яА-ЯїЇєЄіIґҐ]*|[^а-яА-Я0-9їЇєЄіIґҐ]'
        no_letter_or_didgit_pattern = r'[^а-яА-Я0-9їЇєЄіIґҐ]'
        tokens = re.findall(pattern, input_text)
        if SHOW_TOKENS:
            print(tokens)

        if tokens:
            # Handle the first word separately
            first_word = tokens[0]
            compressed_tokens.append(first_word)
            current_text += first_word
            tokens = tokens[1:]

        for token in tokens:
            if re.fullmatch(no_letter_or_didgit_pattern, token):
                # If the token is spaces or a special symbol, append it as is
                compressed_tokens.append(token)
            elif token.isdigit():
                # If the token is a number, simply append it with a prefix "nu"
                compressed_tokens.append("<n" + token+">")
            else:
                # Predict the next words based on the current text
                prediction = self.predict_next(current_text, NUM_WORDS)
                if SHOW_PREDICTION:
                        print(prediction)

                if token in prediction:
                    # If the token is predicted, append its index as a number
                    index = prediction.index(token)
                    compressed_tokens.append(str(index))
                    if SHOW_PREDICTION:
                        print(index)
                else:
                    # If the token is not predicted, append it as is
                    compressed_tokens.append(token)
                    if SHOW_PREDICTION:
                        print(token)

            # Update the current_text
            current_text += token

        # Join the compressed tokens into a single string
        compressed_text = "".join(compressed_tokens)
        return compressed_text

    def decompress(self, compressed_text):
        decompressed_tokens = []
        current_text = ""
        add_number = False  # Flag to indicate the next number should be added as it is

        # Use regular expression to split the compressed text into tokens
        pattern = r'<n\d+>|\d+|[а-яА-ЯїЇєЄіIґҐ]+\'?[а-яА-ЯїЇєЄіIґҐ]*|[^а-яА-Я0-9їЇєЄіIґҐ]'
        number_pattern = r'<n\d+>'
        tokens = re.findall(pattern, compressed_text)
        if SHOW_TOKENS:
            print(tokens)

        for token in tokens:
            if re.fullmatch(number_pattern, token):
                # If the token matches the number pattern add it as a regular number
                token = token[2:len(token)-1]
                decompressed_tokens.append(token)
            elif token.isdigit():
                # Convert the number to an integer
                word_index = int(token)
                # Predict the next words based on the current text
                prediction = self.predict_next(current_text, NUM_WORDS)

                # Use the predicted word at the specified index
                if word_index < len(prediction):
                    word = prediction[word_index]
                    # Check if the word is a special character and adjust spacing accordingly
                    decompressed_tokens.append(word)
                else:
                    # If the index is out of range, append the number itself
                    decompressed_tokens.append(token)
            else:
                # If the token is not a number, simply append it to the current text
                decompressed_tokens.append(token)

            # Update the current_text
            current_text = "".join(decompressed_tokens)

        # Join the decompressed tokens into a single string
        decompressed_text = "".join(decompressed_tokens)
        return decompressed_text

    def predict_next(self, input_text, num_words):
        input_text = input_text[-PREDICT_WINDOW:]
        input_ids = self.tokenizer.encode(input_text, add_special_tokens=False, return_tensors='pt')

        # Generate probabilities for the next word
        with torch.no_grad():
            logits = self.model(input_ids).logits

        # Get the last token's probabilities
        next_word_logits = logits[:, -1, :]

        # Get the top num_words words with the highest probabilities
        top_next_word_indices = torch.topk(next_word_logits, k=num_words, dim=-1).indices[0]
        top_next_words = [self.tokenizer.decode(idx.item()) for idx in top_next_word_indices]
        if SHOW_PREDICTION:
            print(top_next_words)
        return top_next_words


In [ ]:
# Example usage:
model_name = "Tereveni-AI/gpt2-124M-uk-fiction"
text_compressor = TextCompressor(model_name)

while (True):
    command = input('>>>')
    if command == "stop":
        break
    if command == "help":
        print("stop\ncompress file_name\ndecompress file_name")
        continue
    filename = input('Enter a file name: ')
    try:
        in_file = open(filename, 'r')
    except FileNotFoundError:
        print("FileNotFoundError: No such file or directory", filename)
        continue
    except PermissionError:
        print("Permission Denied Error: Access is denied")
        continue
    else:
        input_text = in_file.read()
        print("Input text: ", input_text)
    finally:
        in_file.close()

    input_size = len(input_text)
    if command == "compress":
        with open("(compressed)" + filename, 'w') as out_file:
            compressed_text = text_compressor.compress(input_text)
            print("Compressed text: ", compressed_text)
            out_file.write(compressed_text)
            output_size = len(compressed_text)
            print("Compressing status: {:%}".format((input_size - output_size)/input_size))

    if command == "decompress":
        with open("(decompressed)" + filename, 'w') as out_file:
            decompressed_text = text_compressor.decompress(input_text)
            print("Decompressed text: ", decompressed_text)
            out_file.write(decompressed_text)


>>>decompress
Enter a file name: (compressed)test.txt
Input text:  Верблюд — 6 жвачний ссавець ряду парнокопитних. 2 2 0 0 жвачних верблюд має трьохкамерний 87. Має 5 9 2 30 9 85 33’1 підошвою, пристосованою 1 51 0 3, 0 копита, 10 0 нігті.
Тварини тривалий 0 35 обходитися 0 0, 1 6 випаде 0 знайти 7 6 колючу рослину (16 29 0 1 — верблюжа колючка), 0 76 0 зжує, 5 їхній 17 нечутливий 0 уколів 2 захищений 0 0. Верблюди довго можуть обходитися 0 1, 24 13 натраплять, 0 випивають 22 <n10> 1 2 <n130> літрів. Темного 0 16 82 57 15, 0 27 4 0 32 <n1> кілометру. Довга пухнаста вовна 94 44 0 перегрівання 9 палючим 0 7.
Decompressed text:  Верблюд — великий жвачний ссавець ряду парнокопитних. На відміну від інших жвачних верблюд має трьохкамерний шлунок. Має два пальці на нозі з широкою м’якою підошвою, пристосованою для ходіння по піску, і копита, схожі на нігті.
Тварини тривалий час можуть обходитися без їжі, а якщо випаде нагода знайти в пустелі колючу рослину (вона так і називається — верблюжа к